# SVM - One Vs All Method  - Building 7 Models 

In [1]:
import os.path
import pandas as pd
import numpy as np 

from sklearn.cross_validation import train_test_split
from sklearn.datasets import load_digits
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer, \
    hamming_loss
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, Imputer
from sklearn.svm import SVC

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
DATA_DIR = os.path.join("tree_FINAL")
tree_data = pd.read_csv(DATA_DIR, index_col=0)
pd.set_option('display.max_columns', 1000)

In [3]:
#let's drop some of the columns we won't need

tree_data.drop(['soil_type', 'WA_type'], axis=1, inplace=True)

In [4]:
tree_data.head()

,Elevation,Aspect,Slope,hDist_Water,vDist_Water,hDist_Road,shade_9am,shade_noon,shade_3pm,hDist_Fire,WA1,WA2,WA3,WA4,ST1,ST2,ST3,ST4,ST5,ST6,ST8,ST9,ST10,ST11,ST12,ST13,ST14,ST16,ST17,ST18,ST19,ST20,ST21,ST22,ST23,ST24,ST25,ST26,ST27,ST28,ST29,ST30,ST31,ST32,ST33,ST34,ST35,ST36,ST37,ST38,ST39,ST40,Tree_Type
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
2,2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
3,2804,139,9,268,65,3180,234,238,135,6121,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
4,2785,155,18,242,118,3090,238,238,122,6211,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2
5,2595,45,2,153,-1,391,220,234,150,6172,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5


In [5]:
tree_data.shape

(15120, 53)

# This is my hold out test set to validate that my own model > built in model

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

tree_data, tree_data_test = train_test_split(tree_data, test_size=0.2)

In [7]:
#use tree_data_test to verify which model does best!!!

In [8]:
tree_data_test.to_csv('tree_data_FINALTEST.csv')

In [9]:
tree_data.shape
tree_data.to_csv('tree_data_FINALTRAIN.csv')

# Model 1:

# 1 (Tree Type = 1) Vs 0 (Tree Type = 2,3,4,5,6,7) - Make sure classes are BALANCED!

In [10]:
#create a copy of the original dataframe
tree_data1 = tree_data.copy()

In [11]:
#set Tree_Type =1 --> 1 and all else to 0 for Tree_Type column
tree_data1.Tree_Type = tree_data1.Tree_Type.replace(2, 0)
tree_data1.Tree_Type = tree_data1.Tree_Type.replace(3, 0)
tree_data1.Tree_Type = tree_data1.Tree_Type.replace(4, 0)
tree_data1.Tree_Type = tree_data1.Tree_Type.replace(5, 0)
tree_data1.Tree_Type = tree_data1.Tree_Type.replace(6, 0)
tree_data1.Tree_Type = tree_data1.Tree_Type.replace(7, 0)

In [12]:
tree_data1.Tree_Type.value_counts()

0    10385
1     1711
Name: Tree_Type, dtype: int64

**i want the 0 value counts to be 1738 - BALANCING MY CLASSES OUT!**

In [13]:
from sklearn.utils import resample

#BALANCING MY CLASSES

# Separate majority and minority classes
t_data_0 = tree_data1[tree_data1.Tree_Type==0]
t_data_1 = tree_data1[tree_data1.Tree_Type==1]
 
# Downsample majority class
t_data_0_downsampled = resample(t_data_0, replace=False, n_samples=1725,random_state=1234) 
 
# Combine minority class with downsampled majority class
t_data = pd.concat([t_data_0_downsampled, t_data_1])
 
# Display new class counts
t_data.Tree_Type.value_counts()

0    1725
1    1711
Name: Tree_Type, dtype: int64

### Choosing Predictor and Response Variables 

In [14]:
X_train = t_data.iloc[:, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51]]
y_train = t_data.Tree_Type

### SVM Pipeline

In [15]:

from sklearn.cross_validation import train_test_split
from sklearn.datasets import load_digits
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer, \
    hamming_loss
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, Imputer
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [16]:
#create pipeline
pipe = Pipeline([('scale', StandardScaler(with_mean=True, with_std=True)), ('svm', SVC(probability=True))])
#set param_grid
param_grid = {'svm__C':[3,6,9,12,15, 50, 100, 1000], 'svm__gamma':[0.01, 0.05, 0.1, 0.5, 1]}
#grid search through the parameters 
grid = GridSearchCV(pipe, cv=2, param_grid=param_grid, scoring='accuracy', refit=True)
#fit it to my data 
grid.fit(X_train, y_train)

GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svm', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'svm__C': [3, 6, 9, 12, 15, 50, 100, 1000], 'svm__gamma': [0.01, 0.05, 0.1, 0.5, 1]},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [17]:
grid.best_params_

{'svm__C': 50, 'svm__gamma': 0.01}

### Save this model

In [18]:
#save this model! can load it from here :)


import pickle
with open('model1_svm_onevsall.pickle', 'wb') as handle:
    pickle.dump(grid, handle)

In [19]:
#to open:


#with open('model1_svm_onevsall.pickle', "rb") as input_file1:
    #model1 = pickle.load(input_file1)
    
#-----------------------------------------------------------------
    
#to use:

#y_pred = model1.predict(X_test)

# Model 2:

# 1 (Tree_Type = 2), 0  (Tree Type = 1,3,4,5,6,7)

In [20]:
#create a copy of the original dataframe
tree_data2 = tree_data.copy()

In [21]:
#set Tree_Type =2 --> 1 and all else to 0
tree_data2.Tree_Type = tree_data2.Tree_Type.replace(1, 0)
tree_data2.Tree_Type = tree_data2.Tree_Type.replace(2, 1)
tree_data2.Tree_Type = tree_data2.Tree_Type.replace(3, 0)
tree_data2.Tree_Type = tree_data2.Tree_Type.replace(4, 0)
tree_data2.Tree_Type = tree_data2.Tree_Type.replace(5, 0)
tree_data2.Tree_Type = tree_data2.Tree_Type.replace(6, 0)
tree_data2.Tree_Type = tree_data2.Tree_Type.replace(7, 0)

In [22]:
tree_data2.Tree_Type.value_counts()

0    10364
1     1732
Name: Tree_Type, dtype: int64

In [23]:
#BALANCING MY CLASSES

# Separate majority and minority classes
t_data_0 = tree_data2[tree_data2.Tree_Type==0]
t_data_1 = tree_data2[tree_data2.Tree_Type==1]
 
# Downsample majority class
t_data_0_downsampled = resample(t_data_0, replace=False, n_samples=1693,random_state=1234) 
 
# Combine minority class with downsampled majority class
t_data = pd.concat([t_data_0_downsampled, t_data_1])
 
# Display new class counts
t_data.Tree_Type.value_counts()

1    1732
0    1693
Name: Tree_Type, dtype: int64

### Choosing Predictor and Response Variables 

In [24]:
X_train2 = t_data.iloc[:, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51]]
y_train2 = t_data.Tree_Type

### SVM Pipeline

In [25]:
#create pipeline
pipe = Pipeline([('scale', StandardScaler(with_mean=True, with_std=True)), ('svm', SVC(probability=True))])
#set param_grid
param_grid = {'svm__C':[3,6,9,12,15, 50, 100, 1000], 'svm__gamma':[0.01, 0.05, 0.1, 0.5, 1]}
#grid search through the parameters 
grid2 = GridSearchCV(pipe, cv=2, param_grid=param_grid, scoring='accuracy', refit=True)
#fit it to my data 
grid2.fit(X_train2, y_train2)

GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svm', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'svm__C': [3, 6, 9, 12, 15, 50, 100, 1000], 'svm__gamma': [0.01, 0.05, 0.1, 0.5, 1]},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [26]:
grid2.best_params_

{'svm__C': 9, 'svm__gamma': 0.05}

### Save this model

In [27]:
#import pickle
with open('model2_svm_onevsall.pickle', 'wb') as handle:
    pickle.dump(grid2, handle)

In [28]:
#to open:


#with open('model2_svm_onevsall.pickle', "rb") as input_file2:
    #model2 = pickle.load(input_file2)
    
#-----------------------------------------------------------------
    
#to use:

#y_pred = model2.predict(X_test)

# Model 3 

# 1 (Tree_Type = 3), 0 (Tree Type = 1,2,4,5,6,7)

In [29]:
#create a copy of the original dataframe
tree_data3 = tree_data.copy()

In [30]:
#set Tree_Type =3 --> 1 and all else to 0
tree_data3.Tree_Type = tree_data3.Tree_Type.replace(1, 0)
tree_data3.Tree_Type = tree_data3.Tree_Type.replace(2, 0)
tree_data3.Tree_Type = tree_data3.Tree_Type.replace(3, 1)
tree_data3.Tree_Type = tree_data3.Tree_Type.replace(4, 0)
tree_data3.Tree_Type = tree_data3.Tree_Type.replace(5, 0)
tree_data3.Tree_Type = tree_data3.Tree_Type.replace(6, 0)
tree_data3.Tree_Type = tree_data3.Tree_Type.replace(7, 0)

In [31]:
tree_data3.Tree_Type.value_counts()

0    10392
1     1704
Name: Tree_Type, dtype: int64

In [32]:
#BALANCING MY CLASSES

# Separate majority and minority classes
t_data_0 = tree_data3[tree_data3.Tree_Type==0]
t_data_1 = tree_data3[tree_data3.Tree_Type==1]
 
# Downsample majority class
t_data_0_downsampled = resample(t_data_0, replace=False, n_samples=1766,random_state=1234) 
 
# Combine minority class with downsampled majority class
t_data = pd.concat([t_data_0_downsampled, t_data_1])
 
# Display new class counts
t_data.Tree_Type.value_counts()

0    1766
1    1704
Name: Tree_Type, dtype: int64

### Choosing Predictor and Response Variables

In [33]:
X_train3 = t_data.iloc[:, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51]]
y_train3 = t_data.Tree_Type

### SVM Pipeline

In [34]:
#create pipeline
pipe = Pipeline([('scale', StandardScaler(with_mean=True, with_std=True)), ('svm', SVC(probability=True))])
#set param_grid
param_grid = {'svm__C':[3,6,9,12,15, 50, 100, 1000], 'svm__gamma':[0.01, 0.05, 0.1, 0.5, 1]}
#grid search through the parameters 
grid3 = GridSearchCV(pipe, cv=2, param_grid=param_grid, scoring='accuracy', refit=True)
#fit it to my data 
grid3.fit(X_train3, y_train3)

GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svm', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'svm__C': [3, 6, 9, 12, 15, 50, 100, 1000], 'svm__gamma': [0.01, 0.05, 0.1, 0.5, 1]},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [35]:
grid3.best_params_

{'svm__C': 1000, 'svm__gamma': 0.01}

### Save this model

In [36]:
#import pickle
with open('model3_svm_onevsall.pickle', 'wb') as handle:
    pickle.dump(grid3, handle)

In [37]:
#to open:


#with open('model3_svm_onevsall.pickle', "rb") as input_file3:
    #model3 = pickle.load(input_file3)
    
#-----------------------------------------------------------------
    
#to use:

#y_pred = model3.predict(X_test)

# Model 4

# 1 (Tree_Type = 4), 0 (Tree Type = 1,2,3,5,6,7)

In [38]:
#create a copy of the original dataframe
tree_data4 = tree_data.copy()

In [39]:
#set Tree_Type =4 --> 1 and all else to 0
tree_data4.Tree_Type = tree_data4.Tree_Type.replace(1, 0)
tree_data4.Tree_Type = tree_data4.Tree_Type.replace(2, 0)
tree_data4.Tree_Type = tree_data4.Tree_Type.replace(3, 0)
tree_data4.Tree_Type = tree_data4.Tree_Type.replace(4, 1)
tree_data4.Tree_Type = tree_data4.Tree_Type.replace(5, 0)
tree_data4.Tree_Type = tree_data4.Tree_Type.replace(6, 0)
tree_data4.Tree_Type = tree_data4.Tree_Type.replace(7, 0)

In [40]:
tree_data4.Tree_Type.value_counts()

0    10365
1     1731
Name: Tree_Type, dtype: int64

In [41]:
#BALANCING MY CLASSES

# Separate majority and minority classes
t_data_0 = tree_data4[tree_data4.Tree_Type==0]
t_data_1 = tree_data4[tree_data4.Tree_Type==1]
 
# Downsample majority class
t_data_0_downsampled = resample(t_data_0, replace=False, n_samples=1740,random_state=1234) 
 
# Combine minority class with downsampled majority class
t_data = pd.concat([t_data_0_downsampled, t_data_1])
 
# Display new class counts
t_data.Tree_Type.value_counts()

0    1740
1    1731
Name: Tree_Type, dtype: int64

### Choosing Predictor and Response Variables

In [42]:
X_train4 = t_data.iloc[:, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51]]
y_train4 = t_data.Tree_Type

### SVM Pipeline

In [43]:
#create pipeline
pipe = Pipeline([('scale', StandardScaler(with_mean=True, with_std=True)), ('svm', SVC(probability=True))])
#set param_grid
param_grid = {'svm__C':[3,6,9,12,15, 50, 100, 1000], 'svm__gamma':[0.01, 0.05, 0.1, 0.5, 1]}
#grid search through the parameters 
grid4 = GridSearchCV(pipe, cv=2, param_grid=param_grid, scoring='accuracy', refit=True)
#fit it to my data 
grid4.fit(X_train4, y_train4)

GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svm', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'svm__C': [3, 6, 9, 12, 15, 50, 100, 1000], 'svm__gamma': [0.01, 0.05, 0.1, 0.5, 1]},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [44]:
grid4.best_params_

{'svm__C': 12, 'svm__gamma': 0.1}

In [45]:
#import pickle
with open('model4_svm_onevsall.pickle', 'wb') as handle:
    pickle.dump(grid4, handle)

In [46]:
#to open:


#with open('model4_svm_onevsall.pickle', "rb") as input_file4:
    #model4 = pickle.load(input_file4)
    
#-----------------------------------------------------------------
    
#to use:

#y_pred = model4.predict(X_test)

# Model 5

# 1 (Tree_Type = 5), 0 (Tree Type = 1,2,3,4,6,7)

In [47]:
#create a copy of the original dataframe
tree_data5 = tree_data.copy()

In [48]:
#set Tree_Type =5 --> 1 and all else to 0
tree_data5.Tree_Type = tree_data5.Tree_Type.replace(1, 0)
tree_data5.Tree_Type = tree_data5.Tree_Type.replace(2, 0)
tree_data5.Tree_Type = tree_data5.Tree_Type.replace(3, 0)
tree_data5.Tree_Type = tree_data5.Tree_Type.replace(4, 0)
tree_data5.Tree_Type = tree_data5.Tree_Type.replace(5, 1)
tree_data5.Tree_Type = tree_data5.Tree_Type.replace(6, 0)
tree_data5.Tree_Type = tree_data5.Tree_Type.replace(7, 0)

In [49]:
tree_data5.Tree_Type.value_counts()

0    10355
1     1741
Name: Tree_Type, dtype: int64

In [50]:
#BALANCING MY CLASSES

# Separate majority and minority classes
t_data_0 = tree_data5[tree_data5.Tree_Type==0]
t_data_1 = tree_data5[tree_data5.Tree_Type==1]
 
# Downsample majority class
t_data_0_downsampled = resample(t_data_0, replace=False, n_samples=1723,random_state=1234) 
 
# Combine minority class with downsampled majority class
t_data = pd.concat([t_data_0_downsampled, t_data_1])
 
# Display new class counts
t_data.Tree_Type.value_counts()

1    1741
0    1723
Name: Tree_Type, dtype: int64

### Choosing Predictor and Response Variables

In [51]:
X_train5 = t_data.iloc[:, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51]]
y_train5 = t_data.Tree_Type

### SVM Pipeline

In [52]:
#create pipeline
pipe = Pipeline([('scale', StandardScaler(with_mean=True, with_std=True)), ('svm', SVC(probability=True))])
#set param_grid
param_grid = {'svm__C':[3,6,9,12,15, 50, 100, 1000], 'svm__gamma':[0.01, 0.05, 0.1, 0.5, 1]}
#grid search through the parameters 
grid5 = GridSearchCV(pipe, cv=2, param_grid=param_grid, scoring='accuracy', refit=True)
#fit it to my data 
grid5.fit(X_train5, y_train5)

GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svm', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'svm__C': [3, 6, 9, 12, 15, 50, 100, 1000], 'svm__gamma': [0.01, 0.05, 0.1, 0.5, 1]},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [53]:
grid5.best_params_

{'svm__C': 50, 'svm__gamma': 0.05}

### Save this model

In [54]:
#import pickle
with open('model5_svm_onevsall.pickle', 'wb') as handle:
    pickle.dump(grid5, handle)

In [55]:
#to open:


#with open('model5_svm_onevsall.pickle', "rb") as input_file5:
    #model5 = pickle.load(input_file5)
    
#-----------------------------------------------------------------
    
#to use:

#y_pred = model5.predict(X_test)

# Model 6

# 1 (Tree_Type = 6), 0 (Tree Type = 1,2,3,4,5,7)

In [56]:
#create a copy of the original dataframe
tree_data6 = tree_data.copy()

In [57]:
#set Tree_Type =6 --> 1 and all else to 0
tree_data6.Tree_Type = tree_data6.Tree_Type.replace(1, 0)
tree_data6.Tree_Type = tree_data6.Tree_Type.replace(2, 0)
tree_data6.Tree_Type = tree_data6.Tree_Type.replace(3, 0)
tree_data6.Tree_Type = tree_data6.Tree_Type.replace(4, 0)
tree_data6.Tree_Type = tree_data6.Tree_Type.replace(5, 0)
tree_data6.Tree_Type = tree_data6.Tree_Type.replace(6, 1)
tree_data6.Tree_Type = tree_data6.Tree_Type.replace(7, 0)

In [58]:
tree_data6.Tree_Type.value_counts()

0    10348
1     1748
Name: Tree_Type, dtype: int64

In [59]:
#BALANCING MY CLASSES

# Separate majority and minority classes
t_data_0 = tree_data6[tree_data6.Tree_Type==0]
t_data_1 = tree_data6[tree_data6.Tree_Type==1]
 
# Downsample majority class
t_data_0_downsampled = resample(t_data_0, replace=False, n_samples=1726,random_state=1234) 
 
# Combine minority class with downsampled majority class
t_data = pd.concat([t_data_0_downsampled, t_data_1])
 
# Display new class counts
t_data.Tree_Type.value_counts()

1    1748
0    1726
Name: Tree_Type, dtype: int64

### Choosing Predictor and Response Variables

In [60]:
X_train6 = t_data.iloc[:, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51]]
y_train6 = t_data.Tree_Type

### SVM Pipeline

In [61]:
#create pipeline
pipe = Pipeline([('scale', StandardScaler(with_mean=True, with_std=True)), ('svm', SVC(probability=True))])
#set param_grid
param_grid = {'svm__C':[3,6,9,12,15, 50, 100, 1000], 'svm__gamma':[0.01, 0.05, 0.1, 0.5, 1]}
#grid search through the parameters 
grid6 = GridSearchCV(pipe, cv=2, param_grid=param_grid, scoring='accuracy', refit=True)
#fit it to my data 
grid6.fit(X_train6, y_train6)


GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svm', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'svm__C': [3, 6, 9, 12, 15, 50, 100, 1000], 'svm__gamma': [0.01, 0.05, 0.1, 0.5, 1]},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [62]:
grid6.best_params_

{'svm__C': 9, 'svm__gamma': 0.05}

### Save this model

In [63]:
#import pickle
with open('model6_svm_onevsall.pickle', 'wb') as handle:
    pickle.dump(grid6, handle)

In [64]:
#to open:


#with open('model6_svm_onevsall.pickle', "rb") as input_file6:
    #model6 = pickle.load(input_file6)
    
#-----------------------------------------------------------------
    
#to use:

#y_pred = model6.predict(X_test)

# Model 7

# 1 (Tree_Type = 7), 0 (Tree Type = 1,2,3,4,5,6)

In [65]:
#create a copy of the original dataframe
tree_data7 = tree_data.copy()

In [66]:
#set Tree_Type =7 --> 1 and all else to 0
tree_data7.Tree_Type = tree_data7.Tree_Type.replace(1, 0)
tree_data7.Tree_Type = tree_data7.Tree_Type.replace(2, 0)
tree_data7.Tree_Type = tree_data7.Tree_Type.replace(3, 0)
tree_data7.Tree_Type = tree_data7.Tree_Type.replace(4, 0)
tree_data7.Tree_Type = tree_data7.Tree_Type.replace(5, 0)
tree_data7.Tree_Type = tree_data7.Tree_Type.replace(6, 0)
tree_data7.Tree_Type = tree_data7.Tree_Type.replace(7, 1)

In [67]:
tree_data7.Tree_Type.value_counts()

0    10367
1     1729
Name: Tree_Type, dtype: int64

In [68]:
#BALANCING MY CLASSES

# Separate majority and minority classes
t_data_0 = tree_data7[tree_data7.Tree_Type==0]
t_data_1 = tree_data7[tree_data7.Tree_Type==1]
 
# Downsample majority class
t_data_0_downsampled = resample(t_data_0, replace=False, n_samples=1723,random_state=1234) 
 
# Combine minority class with downsampled majority class
t_data = pd.concat([t_data_0_downsampled, t_data_1])
 
# Display new class counts
t_data.Tree_Type.value_counts()

1    1729
0    1723
Name: Tree_Type, dtype: int64

### Choosing Predictor and Response Variables

In [69]:
X_train7 = t_data.iloc[:, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51]]
y_train7 = t_data.Tree_Type

### SVM Pipeline

In [70]:
#create pipeline
pipe = Pipeline([('scale', StandardScaler(with_mean=True, with_std=True)), ('svm', SVC(probability=True))])
#set param_grid
param_grid = {'svm__C':[3,6,9,12,15, 50, 100, 1000], 'svm__gamma':[0.01, 0.05, 0.1, 0.5, 1]}
#grid search through the parameters 
grid7 = GridSearchCV(pipe, cv=2, param_grid=param_grid, scoring='accuracy', refit=True)
#fit it to my data 
grid7.fit(X_train7, y_train7)

GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svm', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'svm__C': [3, 6, 9, 12, 15, 50, 100, 1000], 'svm__gamma': [0.01, 0.05, 0.1, 0.5, 1]},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [71]:
grid7.best_params_

{'svm__C': 100, 'svm__gamma': 0.01}

### Save this model

In [72]:
import pickle
with open('model7_svm_onevsall.pickle', 'wb') as handle:
    pickle.dump(grid7, handle)

In [73]:
#to open:


#with open('model7_svm_onevsall.pickle', "rb") as input_file7:
    #model7 = pickle.load(input_file7)
    
#-----------------------------------------------------------------
    
#to use:

#y_pred = model7.predict(X_test)